In [1]:
import scipy
import operator
import numpy as np
import pandas as pd
import cPickle as pickle
from scipy import stats
import seaborn as sns
from datetime import datetime
from collections import Counter
import matplotlib.pyplot as plt
from pprint import pprint
from __future__ import division
from sklearn.utils import resample
import statsmodels.api as sm
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
plt.style.use('classic')
%matplotlib inline

'''
    Analysis originaly performed in Python 2 (deprecated)
    Seaborn, Statsmodel, and * imports broken in Python 3
'''

In [2]:
def cov(arr):
    return np.std(arr) / np.mean(arr) 

def fundingTime(arr):
    return (arr.max() - arr.min()).days

def interEventTime(arr):
    return [(t - s).seconds for s, t in zip(sorted(arr), sorted(arr)[1:])]

def avgInterEventTime(arr):
    return np.mean(interEventTime(arr))

def covInterEventTime(arr):
    varr = interEventTime(arr)
    return np.std(varr) / np.mean(varr) 

In [3]:
bids = pd.read_csv("../Data/diag/bid_notick.txt", sep="|")

In [4]:
bids.columns

Index([u'Amount', u'CreationDate', u'Bid_Key', u'ListingKey', u'ListingStatus', u'MemberKey', u'MinimumRate', u'MinimumYield', u'ParticipationAmount', u'Status', u'ModifiedDate', u'Unnamed: 11'], dtype='object')

In [5]:
Counter(bids['Status'])

Counter({'Bid Withdrawn': 79,
         'Outbid': 2813957,
         'Partially Participating': 46516,
         'Winning': 6872104})

In [6]:
bids.sample(10)

,Amount,CreationDate,Bid_Key,ListingKey,ListingStatus,MemberKey,MinimumRate,MinimumYield,ParticipationAmount,Status,ModifiedDate,Unnamed: 11
4549270,50.000,2008-05-05 22:11:07,F69E3522990329898482A20,EB033419407345143A257DA,Cancelled,989234200912208133857AB,nan,nan,50.000,Winning,2008-05-05 22:11:07,nan
1413614,50.000,2007-04-16 02:56:45,DAA43489818107151EC1773,41B43386126607856C398C7,Completed,BFBE33852659890967FB61E,0.140,0.130,0.000,Outbid,2007-04-20 17:45:14,nan
7389093,50.000,2010-08-25 16:32:50,DB083596353202495F9F4B0,ED2A3491863637528F21E95,Completed,DEB5345942729531225D7AB,0.151,0.141,0.000,Outbid,2010-08-26 01:45:16,nan
3204660,50.000,2007-12-19 09:52:47,65593512808963265F77B3A,AAE43407983601849ABF704,Expired,BBCB3386466236363EEA0E6,nan,nan,50.000,Winning,2007-12-19 09:52:47,nan
3065671,50.000,2007-12-19 11:54:52,8ED63512493652487DA3E17,CACD3408559182088C54BA9,Cancelled,E23C34069888860080E727D,nan,nan,50.000,Winning,2007-12-19 11:54:52,nan
6536878,25.000,2009-12-22 17:28:08,3D333576356015504783CC2,D395347237105340465712F,Cancelled,79633375655131203E74041,nan,nan,25.000,Winning,2009-12-22 17:28:08,nan
3540557,50.000,2008-02-27 00:28:06,7D4F3518309817218D43F75,031934143367901093B8CB9,Expired,F1733409787396377DB6A09,nan,nan,50.000,Winning,2008-02-27 00:28:06,nan
6098390,50.000,2009-10-08 15:49:24,24193568013978092144A8D,26C6346390014236042DD8B,Completed,AFB3340251565581019BA8C,nan,nan,50.000,Winning,2009-10-08 15:49:24,nan
6375269,25.000,2009-11-25 14:09:02,9B1F3572965040599504A8C,1BD234690393322135C9821,Withdrawn,AF3533744432966217FA6B7,nan,nan,25.000,Winning,2009-11-25 14:09:02,nan
3476530,179.400,2008-02-03 11:56:55,A89335160614334035F63A9,BE0C3412557895006C7A55A,Withdrawn,C5713366430455414BEC916,nan,nan,179.400,Winning,2008-02-09 18:47:52,nan


In [7]:
# Filter winning bids
bids = bids[bids['Status'].isin(["Winning", "Partially Participating"])]
bids.sample(10)

,Amount,CreationDate,Bid_Key,ListingKey,ListingStatus,MemberKey,MinimumRate,MinimumYield,ParticipationAmount,Status,ModifiedDate,Unnamed: 11
7178343,25.000,2010-06-28 11:40:41,B8BD3591937298360E8C9B6,4AE03486813894043010C99,Withdrawn,CB193404531126035E27546,nan,nan,25.000,Winning,2010-06-28 11:40:41,nan
5762234,50.000,2008-09-18 11:19:20,8F5635366274451882E5013,D3873431726263863FE5905,Withdrawn,DAA033709857325314423BC,nan,nan,50.000,Winning,2008-09-18 11:19:20,nan
9487746,25.000,2012-01-26 08:05:26,07293641180280890C5232A,71CC35368632805155415FD,Completed,43C53522234125763513685,nan,nan,25.000,Winning,2012-01-26 08:05:26,nan
9497338,50.000,2012-01-30 17:17:17,149A364271031051573A77B,398B353788290295565FA13,Cancelled,7AA7341804288943869FADB,nan,nan,50.000,Winning,2012-01-30 17:17:17,nan
5869660,50.000,2008-09-29 16:39:40,705035370467415250087B5,82E834330278024514FB379,Completed,5C74343198376397409C256,nan,nan,50.000,Winning,2008-09-29 16:39:40,nan
758790,50.000,2007-01-18 07:47:16,892C3379721319236BF5DB8,01C733786269340506C9D0F,Completed,33513377831484554D87A5C,nan,nan,50.000,Winning,2007-01-18 07:47:16,nan
7824635,25.000,2010-12-16 15:57:28,9E7136073362750428D7A02,DE0D3502461645688F9CBA5,Completed,9BF93501889172596AAC0F9,nan,nan,25.000,Winning,2010-12-16 15:57:28,nan
8024070,25.000,2011-02-11 01:09:56,367C36118893922266C1B3E,AF9D350670964381731961B,Completed,68223415171747330AD3006,nan,nan,25.000,Winning,2011-02-11 01:09:56,nan
6861489,100.000,2010-03-22 17:33:49,A8A33582863506041E9EB55,1A253479090933187E0718C,Completed,60C734714362996094B2E19,nan,nan,100.000,Winning,2010-03-22 17:33:49,nan
4017935,50.000,2008-03-20 00:13:41,660335205763703330E1476,B9773415326237090FF6F50,Completed,BA1F3404722968722146C31,nan,nan,50.000,Winning,2008-03-20 00:13:41,nan


In [8]:
Counter(bids['Status'])

Counter({'Partially Participating': 46516, 'Winning': 6872104})

In [9]:
# Remove current listings
bids = bids[bids.ListingStatus.isin(["Expired", "Completed"])]
bids.sample(10)

,Amount,CreationDate,Bid_Key,ListingKey,ListingStatus,MemberKey,MinimumRate,MinimumYield,ParticipationAmount,Status,ModifiedDate,Unnamed: 11
5965537,25.000,2009-09-09 18:02:13,887E3565404323338FB350F,3AC13461931138838398498,Completed,99B83365533930356F20CA0,nan,nan,25.000,Winning,2009-09-09 18:02:13,nan
4652178,50.000,2008-05-19 00:37:22,57F735251963764192D4035,5249341968507261010D61E,Completed,13B63401386068304581CBD,nan,nan,50.000,Winning,2008-05-19 00:37:22,nan
405207,50.000,2006-10-02 08:43:37,606833695850568655C7F47,16943369511052890A68329,Completed,6CC1336629826656743837B,nan,nan,50.000,Winning,2006-10-02 08:43:37,nan
3934859,100.000,2008-03-12 04:06:32,1D7535194180729542FAD30,FBBC34149166580032D3386,Completed,E1E03414383183251AFB9C6,nan,nan,100.000,Winning,2008-03-12 04:06:32,nan
4903453,50.000,2008-06-13 17:07:41,1E7635268190247643ABDF7,C23D342245830954324FE40,Expired,47C0336577949472753DA0F,nan,nan,50.000,Winning,2008-06-13 17:07:41,nan
3916830,54.120,2008-03-07 21:35:59,7ED635186218960517B6822,3BD634144776221492FBDBE,Completed,07AC3382899290370D20000,nan,nan,54.120,Winning,2008-03-07 21:35:59,nan
4977683,50.000,2008-06-18 13:08:22,0020352744324529935ACAE,16C93423868007288BC6E68,Expired,3717336509444357832AF95,nan,nan,50.000,Winning,2008-06-18 13:08:22,nan
6747832,50.000,2010-02-28 11:09:48,7C01358172930003791794E,CC093477174668495D60B56,Expired,0DFF34160762205618A27B4,nan,nan,50.000,Winning,2010-02-28 11:09:48,nan
9550903,3645.000,2012-03-02 09:10:02,2D633644405799291DFE6C4,25863538327291000D35BE8,Completed,FF4E3514265681759505836,nan,nan,879.370,Partially Participating,2012-03-02 09:10:02,nan
2205766,50.000,2007-08-04 04:55:46,DF313500491763560C15621,252833965755324861ED7AA,Expired,88613365464370600431FB4,nan,nan,50.000,Winning,2007-08-04 04:55:46,nan


In [10]:
Counter(bids['ListingStatus'])

Counter({'Completed': 4031698, 'Expired': 1275931})

In [11]:
# Filter relevant columns
bids = bids[["ListingKey", "Bid_Key", "MemberKey", "CreationDate", "Amount", "ListingStatus"]]
bids.sample(10)

,ListingKey,Bid_Key,MemberKey,CreationDate,Amount,ListingStatus
9132841,A00D3529673733932BCEBD0,342F36360118304483BAD05,B7CA35132136519834AB013,2011-11-14 12:00:03,100.000,Completed
9221796,E97B353167674679501EFDF,08493636442879766FC160D,C50B337385510596952CA28,2011-11-30 17:08:52,25.000,Completed
9545440,B0403539533538401495E77,CCC236432872699344A06DE,4A0134172449554712D1300,2012-02-12 17:47:02,50.000,Completed
6017922,999C3466227562986112670,56AD35719818960658A9FDF,42713416379007912C45C04,2009-11-06 15:17:39,150.000,Completed
2903140,1DF8340424193128647CA55,677D3507649976434D2A5DA,73EC3403030673372297053,2007-11-09 11:01:13,50.000,Expired
3467927,56C834093563796587C6624,207E3514155121780EB20BA,599A340822321136318F8C8,2008-01-08 09:34:05,50.000,Expired
8142148,5D4A3510736053109D73471,33D73614148067019ACFB6A,B98C3365638903020C68AE2,2011-03-29 05:51:28,25.000,Completed
3335283,20C334071766724232040C2,1BCA3510695359125B57D6B,D36434038804317809494DE,2007-11-27 16:38:37,50.000,Completed
7423243,27F73493473288363F263DF,F6D03597262904335CACF27,2D6A3397896365827EF1E81,2010-09-03 19:56:23,70.000,Expired
5625834,67143429061749804DABC8B,6E333535034298219BF3EDB,0DF93392595838039D6B77D,2008-09-01 19:02:51,73.970,Completed


In [12]:
# Convert creation date to datetime format
bids["CreationDate"] = pd.to_datetime(bids.CreationDate)
# Rename status colunmn for consistency with other projects
bids = bids.rename(index=str, columns={"ListingStatus": "Status"})
bids.sample(10)

,ListingKey,Bid_Key,MemberKey,CreationDate,Amount,Status
1171159,B56F3384120020875601C47,2CDF33845681273958E8B67,2A0033662257237471E876C,2007-03-22 17:18:32,100.000,Completed
8086295,F3EE3509010490717B0E55D,B78B3612083509799554DD9,7A703403778823205476546,2011-03-04 10:50:56,50.000,Completed
2516889,4DE13399764089269C29FFA,DC83350458861543372307B,D241337574588381327AD33,2007-09-21 14:59:33,100.000,Completed
9661618,3242354142306253684F3C0,557536441059018748EB60E,CD7D34676366550558575B0,2012-03-06 09:17:05,25.000,Completed
8784142,7132352294219732458FCA0,12D43628603917151314245,62B034719211818415DD1DE,2011-08-26 13:21:38,25.000,Expired
6939702,C3393481829964867B0416F,B1BF35861552431820963BD,5E803374304777986D7C9AE,2010-04-19 15:06:25,25.000,Completed
8741693,775635227677460619A88F3,22C73625267666058F26917,405F33716035025945DDF62,2011-08-01 09:02:40,100.000,Completed
8471663,AC3535160612462930CD1BE,F34C3621643535809EB751F,F0CB35082916011409376D9,2011-06-03 17:37:24,25.550,Completed
9048569,F32035290268351523BE013,CF82363369234410460E46B,145D33910175633050BD59D,2011-10-26 16:30:33,25.000,Completed
5763562,EF40343042032293603E726,4A973534030044136E04653,F5A1341163761456670D7D8,2008-09-12 09:29:10,50.000,Completed


In [13]:
print "Contributors:", bids['MemberKey'].nunique()

Contributors: 62807


In [14]:
# Aggregate Functions
f = {'ListingKey': ['max'],
     'MemberKey':['nunique'], # Contributor Count
     'Bid_Key': ['nunique'], # Contributions Count
     'CreationDate': ['min', avgInterEventTime, covInterEventTime], # First Contribution Timestamp, AvgInterEventTime, CovInterEventTime
     'Amount': ['min', 'mean', 'max', cov] # Minimum, Mean, Maximum, CoV
}

In [15]:
# Aggregate bids at project level
projects = pd.DataFrame(bids.groupby('ListingKey').agg(f).as_matrix())
projects = projects.rename(index=str, columns={0: "NumContributors", 
                                       1: "FirstContributionTimestamp", 
                                       2: "AvgInterEventTime_sec", 
                                       3: "CovInterEventTime", 
                                       4: "MinContributionAmount",
                                       5: "MeanContributionAmount",
                                       6: "MaxContributionAmount", 
                                       7: "CovContributionAmount",
                                       8: "NumContributions",
                                       9: "ProjectID"})
projects.sample(10)

,NumContributors,FirstContributionTimestamp,AvgInterEventTime_sec,CovInterEventTime,MinContributionAmount,MeanContributionAmount,MaxContributionAmount,CovContributionAmount,NumContributions,ProjectID
166740,7,2007-06-05 09:35:26,21688.833,1.187,50.000,67.671,100.000,0.325,7,EC1B339025984540787D8F9
123858,32,2008-06-13 07:07:29,212.194,1.094,50.000,63.128,100.000,0.323,32,AF523422340402366EB58D4
136715,10,2008-05-28 19:42:47,8702.700,1.932,50.000,156.233,499.000,0.890,11,C1BC3420660177522CCF180
72812,156,2008-02-05 16:58:17,4317.550,2.279,50.000,62.605,403.010,0.628,161,6727341228346519838E04D
140610,26,2007-11-12 14:21:08,16586.000,1.513,50.000,61.164,100.000,0.330,27,C730340435117498591F819
29617,2,2007-10-29 13:57:00,17900.000,0.000,50.000,50.000,50.000,0.000,2,29FE3402555601531E914E3
4873,1,2008-05-05 13:24:21,NaN,NaN,50.000,50.000,50.000,0.000,1,06D8341874274790073959F
150227,2,2009-10-14 05:10:51,13494.000,0.886,25.000,71.250,175.000,0.850,4,D4963464496502363D0294B
88667,1,2008-09-15 13:30:36,NaN,NaN,50.030,50.030,50.030,0.000,1,7D6E343155545835178F6BE
58940,2,2008-05-09 10:30:34,24586.000,0.000,50.000,50.000,50.000,0.000,2,53AA3419019106738B6F4B3


In [17]:
sum(projects.NumContributions)

5307629

In [22]:
columns = ["Lst_Key", "AmountFunded", "StartDate", "EndDate", "Status", "BorrowerRate", "Duration", "ProsperScore", "CreditGrade", "DebtToIncomeRatio", "MonthlyLoanPayment", "EstimatedLoss", "IsBorrowerHomeowner"]

In [23]:
listings = pd.read_csv('../Data/diag/listing.txt', sep="|")
listings.columns

Index([u'AmountFunded', u'AmountRemaining', u'AmountRequested', u'BankDraftFeeAnnualRate', u'BidCount', u'BidMaximumRate', u'BorrowerCity', u'BorrowerMaximumRate', u'BorrowerRate', u'BorrowerState', u'Category', u'CreationDate', u'CreditGrade', u'ProsperRating', u'DebtToIncomeRatio', u'Description', u'Duration', u'EndDate', u'FundingOption', u'GroupKey', u'GroupLeaderRewardRate', u'HasVerifiedBankAccount', u'Images', u'IsBorrowerHomeowner', u'Lst_Key', u'LenderRate', u'ListingNumber', u'MemberKey', u'PercentFunded', u'StartDate', u'Status', u'Title', u'ModifiedDate', u'ProductSpecID', u'LoanTermInMonths', u'Term', u'EstimatedLoss', u'LenderYield', u'EffectiveYield', u'EstimatedReturn', u'VerificationStage', u'ActiveProsperLoans', u'TotalProsperLoans', u'ProsperPrincipalBorrowed', u'ProsperPrincipalOutstanding', u'OnTimeProsperPayments', u'ProsperPaymentsLessThanOneMonthLate', u'ProsperPaymentsOneMonthPlusLate', u'TotalProsperPaymentsBilled', u'ProsperScore', u'CreditScoreRangeLower',
 

In [24]:
listings = listings[columns]
listings.sample(10)

,Lst_Key,AmountFunded,StartDate,EndDate,Status,BorrowerRate,Duration,ProsperScore,CreditGrade,DebtToIncomeRatio,MonthlyLoanPayment,EstimatedLoss,IsBorrowerHomeowner
319890,974B34301446995624C7B04,150.000,2008-08-19 18:51:00,2008-08-26 18:52:27,Expired,0.320,7,nan,D,0.200,87.090,nan,False
172216,8EE03399701544070F7F9D9,0.000,2007-09-07 06:41:41,2007-09-11 09:41:24,Withdrawn,0.290,10,nan,D,0.280,1047.640,nan,False
392234,25493504736505396B850D6,4000.000,2011-01-05 10:04:21,2011-01-05 10:31:27,Completed,0.278,14,8.000,NaN,0.220,164.910,0.098,False
379040,208D3489220892806942AFF,3890.150,2010-07-23 14:25:41,2010-07-30 14:28:16,Expired,0.350,7,4.000,NaN,0.060,316.650,0.180,True
23157,613633742426929106D21E6,50.000,2006-12-01 14:26:45,2006-12-11 14:28:25,Expired,0.170,10,nan,E,0.130,41.600,nan,False
344656,572834506537460606BF620,0.000,2009-05-05 19:35:35,2009-05-08 16:29:33,Withdrawn,0.350,14,4.000,NaN,0.390,339.270,0.160,False
34830,F92F33687437549628A7BCE,0.000,2006-09-19 07:35:17,2006-09-24 07:40:21,Withdrawn,0.092,7,nan,HR,0.060,63.830,nan,False
327288,F7D93424521175581E54D68,1000.000,2008-07-08 00:21:52,2008-07-15 00:24:45,Completed,0.270,7,10.000,C,nan,40.830,0.060,True
280272,A8203421123966499F93003,158.210,2008-05-18 08:22:04,2008-05-25 08:25:27,Expired,0.350,7,nan,HR,0.070,67.850,nan,False
325789,EBAB3430624973623B647FB,525.000,2008-09-08 07:49:39,2008-09-13 21:32:05,Withdrawn,0.350,7,nan,C,0.300,339.270,nan,False


In [25]:
listings[listings['Status']=='Expired']

,Lst_Key,AmountFunded,StartDate,EndDate,Status,BorrowerRate,Duration,ProsperScore,CreditGrade,DebtToIncomeRatio,MonthlyLoanPayment,EstimatedLoss,IsBorrowerHomeowner
0,C0933365069571441D6651D,0.000,2006-02-25 05:51:10,2006-03-04 05:53:21,Expired,0.120,7,nan,E,0.142,664.290,nan,NaN
2,9D5E336577848196944F197,0.000,2006-03-15 21:52:45,2006-03-29 21:54:21,Expired,0.080,14,nan,HR,0.169,94.010,nan,NaN
13,F7C433651616107108B5E83,0.000,2006-04-11 00:52:59,2006-04-21 00:57:26,Expired,0.228,10,nan,D,0.194,572.860,nan,NaN
15,AFE53364876098574BEA11A,0.000,2006-04-19 14:20:05,2006-04-26 14:24:40,Expired,0.150,7,nan,NC,nan,167.270,nan,False
16,ABA53364509248269675D6B,0.000,2006-04-24 18:04:52,2006-04-29 18:09:47,Expired,0.150,5,nan,HR,0.026,121.330,nan,False
17,A08233664501505519A588D,0.000,2006-04-24 20:22:04,2006-05-04 20:25:52,Expired,0.237,10,nan,HR,0.089,273.710,nan,False
19,945B33663338623421AF890,0.000,2006-04-26 09:01:26,2006-05-03 09:03:17,Expired,0.207,7,nan,D,0.117,112.610,nan,True
26,F2E43364839722999E27410,0.000,2006-05-01 14:51:08,2006-05-06 14:52:41,Expired,0.237,5,nan,HR,0.167,97.750,nan,False
27,528E3365580878505644E2C,0.000,2006-05-01 21:23:06,2006-05-08 21:25:19,Expired,0.223,7,nan,HR,0.210,71.310,nan,False
28,B1993365440519749A20340,0.000,2006-05-15 16:40:04,2006-05-20 16:43:36,Expired,0.237,5,nan,E,0.170,105.940,nan,False


In [26]:
listings[listings['Status']=='Completed']

,Lst_Key,AmountFunded,StartDate,EndDate,Status,BorrowerRate,Duration,ProsperScore,CreditGrade,DebtToIncomeRatio,MonthlyLoanPayment,EstimatedLoss,IsBorrowerHomeowner
41280,60BE3365719073900A79568,5000.000,2006-02-13 08:33:55,2006-02-20 08:37:33,Completed,0.080,7,nan,AA,0.111,156.680,nan,NaN
41281,94E93366447536288914A23,7500.000,2006-02-15 11:17:23,2006-02-20 11:17:33,Completed,0.080,5,nan,A,0.065,235.020,nan,NaN
41282,94DC3366145318688CD58A8,5000.000,2006-02-13 08:34:24,2006-02-13 08:34:24,Completed,0.090,3,nan,A,0.073,159.000,nan,NaN
41283,8A9B3365100419986A18C9B,10000.000,2006-02-06 23:36:31,2006-02-13 23:36:51,Completed,0.120,7,nan,D,0.183,332.140,nan,NaN
41284,0E0F336443449038617E9F4,8998.000,2005-12-21 19:37:33,2005-12-28 19:40:25,Completed,0.095,7,nan,A,0.070,288.020,nan,NaN
41285,3480336511078238810A782,3000.000,2005-11-18 14:48:19,2005-12-02 14:49:17,Completed,0.060,14,nan,AA,0.019,91.270,nan,NaN
41286,B01233643651103736EE3B6,5000.000,2005-11-18 14:22:34,2005-11-25 14:25:29,Completed,0.085,7,nan,AA,0.027,157.840,nan,NaN
41287,2F123364529418907A58D4C,5000.000,2005-11-18 14:16:59,2005-11-25 14:20:29,Completed,0.085,7,nan,AA,0.015,157.840,nan,NaN
41288,B6E73366056348939D8801F,15000.000,2005-11-20 14:45:55,2005-11-23 09:43:55,Completed,0.120,7,nan,C,0.291,498.210,nan,NaN
41289,F95233659591793039EF09A,4400.000,2006-02-14 04:59:11,2006-02-21 00:17:39,Completed,0.087,7,nan,B,0.280,139.410,nan,NaN


In [36]:
listings[listings['StartDate']>'2009']

,Lst_Key,AmountFunded,StartDate,EndDate,Status,BorrowerRate,Duration,ProsperScore,CreditGrade,DebtToIncomeRatio,MonthlyLoanPayment,EstimatedLoss,IsBorrowerHomeowner
105989,D4903391180910123AECB8E,125.000,2010-06-07 15:59:43,2010-06-14 16:04:29,Expired,0.180,7,7.000,AA,0.170,650.740,0.070,True
133941,80ED33907417258974041D1,3000.000,2009-09-29 16:00:47,2009-10-13 16:04:27,Completed,0.263,14,7.000,C,nan,121.400,0.100,False
147005,B3223392203849125A4DD45,2225.000,2009-12-28 16:01:36,2010-01-04 16:09:29,Expired,0.211,7,9.000,C,0.230,252.700,0.065,True
150922,92133393959392981469BD7,30.000,2010-01-20 16:04:07,2010-01-27 16:22:04,Expired,0.250,7,4.000,C,0.560,198.800,0.180,False
151393,0B0F339369525260679BD6F,0.000,2010-03-18 16:19:29,2010-03-25 16:29:47,Expired,0.350,7,1.000,B,nan,1130.900,0.345,True
151891,4F293394422304293113AA8,22518.160,2009-10-13 15:57:31,2009-10-20 16:04:35,Expired,0.131,7,6.000,AA,0.220,843.910,0.015,True
155696,037C3393451006201C59372,903.000,2010-02-04 16:52:08,2010-02-11 17:01:33,Expired,0.211,7,9.000,B,0.370,101.830,0.065,True
160846,FC5133984213291796D5347,7202.830,2010-09-14 14:27:51,2010-09-21 14:37:20,Expired,0.275,7,6.000,B,nan,616.410,0.108,False
171154,6BA83398632300226A9405E,0.000,2009-12-31 15:54:56,2010-01-07 16:01:35,Expired,0.250,7,2.000,B,nan,497.000,0.250,True
171704,5662339805297736858803F,0.000,2009-09-18 16:13:04,2009-10-02 16:15:51,Expired,0.182,14,1.000,C,nan,543.640,0.345,True


In [35]:
min(listings[listings['StartDate']>'2009']['StartDate'])

'2009-04-28 03:34:06'